In [1]:
import tensorflow as tf
import numpy as np
import vgg16_network as net
import read_img
import pandas as pd
import os
from sklearn.cross_validation import train_test_split

D:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
LEARNING_RATE_BASE = 0.001
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0002
MAX_ITER = 40000
BATCH_SIZE=20

In [3]:
save_checkpoint_dir = '/path/to/save_model/Driver_Detection_ver8/'
read_checkpoint_dir = '/path/to/save_model/Driver_Detection_ver4/'
ckt_name = 'model.ckpt-14010'
#trainc0 ver5 'model.ckpt-14010'
#trainc1 ver6 'model.ckpt-5826'
#trainc2 ver7'model.ckpt-10329'

In [4]:
def train_val_data(data,pre_test=0.1,random_val=True):
    if random_val:
        data_c0 = data[np.where(data[:,1]=='c0')[0]]
        data_c1 = data[np.where(data[:,1]=='c1')[0]]
        data_c2 = data[np.where(data[:,1]=='c2')[0]]
        data_c3 = data[np.where(data[:,1]=='c3')[0]]
        data_c4 = data[np.where(data[:,1]=='c4')[0]]
        data_c5 = data[np.where(data[:,1]=='c5')[0]]
        data_c6 = data[np.where(data[:,1]=='c6')[0]]
        data_c7 = data[np.where(data[:,1]=='c7')[0]]
        data_c8 = data[np.where(data[:,1]=='c8')[0]]
        data_c9 = data[np.where(data[:,1]=='c9')[0]]

        train_c0,test_c0,__,___ = train_test_split(data_c0,np.zeros(data_c0.shape[0]),test_size =pre_test)
        train_c1,test_c1,__,___ = train_test_split(data_c1,np.zeros(data_c1.shape[0]),test_size =pre_test)
        train_c2,test_c2,__,___ = train_test_split(data_c2,np.zeros(data_c2.shape[0]),test_size =pre_test)
        train_c3,test_c3,__,___ = train_test_split(data_c3,np.zeros(data_c3.shape[0]),test_size =pre_test)
        train_c4,test_c4,__,___ = train_test_split(data_c4,np.zeros(data_c4.shape[0]),test_size =pre_test)
        train_c5,test_c5,__,___ = train_test_split(data_c5,np.zeros(data_c5.shape[0]),test_size =pre_test)
        train_c6,test_c6,__,___ = train_test_split(data_c6,np.zeros(data_c6.shape[0]),test_size =pre_test)
        train_c7,test_c7,__,___ = train_test_split(data_c7,np.zeros(data_c7.shape[0]),test_size =pre_test)
        train_c8,test_c8,__,___ = train_test_split(data_c8,np.zeros(data_c8.shape[0]),test_size =pre_test)
        train_c9,test_c9,__,___ = train_test_split(data_c9,np.zeros(data_c9.shape[0]),test_size =pre_test)
        train_data = np.concatenate((train_c0,train_c1,train_c2,train_c3,train_c4,train_c5,train_c6,train_c7,train_c8,train_c9))
        test_data = np.concatenate((test_c0,test_c1,test_c2,test_c3,test_c4,test_c5,test_c6,test_c7,test_c8,test_c9))
    else:
        subject_name = list(set(data[:,0]))
        train_c0,test_c0,__,___ = train_test_split(subject_name,np.zeros(len(subject_name)),test_size =0.15)
        train_c0 = [ 'p042','p012', 'p072', 'p024','p064','p051', 'p081', 'p022',  'p061', 'p066', 'p002', 'p015', 'p052', 'p014', 'p026', 'p056', 'p049', 'p050', 'p035', 'p075', 'p047', 'p039']
        test_c0 = ['p021', 'p041', 'p016', 'p045']
        train_data = data[np.where(data[:,0]==train_c0[0])[0]]
        for i in range(1,len(train_c0)):
            cc = data[np.where(data[:,0]==train_c0[i])[0]]
            train_data = np.concatenate((train_data,cc))

        test_data = data[np.where(data[:,0]==test_c0[0])[0]]
        for i in range(1,len(test_c0)):
            cc = data[np.where(data[:,0]==test_c0[i])[0]]
            test_data = np.concatenate((test_data,cc))
    np.random.shuffle(train_data)
    np.random.shuffle(test_data)
    return train_data,test_data

In [5]:
def train(dataAug=False,checkpoint=False):    
    input_data = tf.placeholder(tf.float32,shape=[BATCH_SIZE,None,None,3],name='img_input')
    label = tf.placeholder(tf.int32,shape=[None],name='label')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    learning_rate = tf.placeholder(tf.float32,name='learning_rate')
    L2_regu_rate = tf.placeholder(tf.float32,name='L2_regu_rate')
    regularizer = tf.contrib.layers.l2_regularizer(L2_regu_rate)
    if dataAug:
        vgg_input = read_img.tfDataAug(input_data,batch_size=BATCH_SIZE)
    else:
        vgg_input = input_data
    cls_10 = net.VGG16(vgg_input,keep_prob,regularizer,checkpoint=checkpoint,checkpoint_dir=read_checkpoint_dir,ckt_name=ckt_name)
    cls_10_softmax = tf.nn.softmax(cls_10)
    cls_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=cls_10,labels=label))
    
    global_step = tf.Variable(0, trainable=False,name="global_step")
    loss = cls_loss + tf.add_n(tf.get_collection('losses'))
    
    #learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,1000, LEARNING_RATE_DECAY,staircase=True)
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    #train_op = tf.train.MomentumOptimizer(learning_rate,MOMENTUM).minimize(loss,global_step=global_step)
    #train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    tf.summary.scalar('all_loss', loss)
    tf.summary.scalar('learning_rate', learning_rate)
    merged = tf.summary.merge_all()
    #################################
    saver = tf.train.Saver()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        summary_writer = tf.summary.FileWriter(r"/path/to/Driver_Detection",sess.graph)
        tf.global_variables_initializer().run()
        if checkpoint:
            n_StartCrossVal=0
        else:
            n_StartCrossVal=5
        count =0
        start = 0
        end = BATCH_SIZE
        #driver_imgs_list = pd.read_csv('data/driver_imgs_list.csv')
        driver_imgs_list = pd.read_csv('data/Aug_driver_imgs_list.csv')
        train_data,test_data = train_val_data(driver_imgs_list.values,pre_test=0.1,random_val=True)
        train_data_len = train_data.shape[0]
        n_batch = train_data_len//BATCH_SIZE
        accury_max = 0
        num_acc_noupdate=0
        pre_train_num = 0
        val_loss = 20
        while count<MAX_ITER:
            if num_acc_noupdate>10:break
            img =[]
            img_label = []
            for i in range(start,end):
                i = i%train_data_len
                img_file = os.path.join('data','train', train_data[i,1]) + '\\'+ train_data[i,2]
                img_tmp = read_img.get_img_sample(img_file,dataAug=dataAug)
                label_tmp = [int(train_data[i,1][1])]
                img.append(img_tmp)
                img_label.append(label_tmp)
            img = np.array(img).reshape(BATCH_SIZE,img_tmp.shape[1],img_tmp.shape[2],3)
            img_label = np.array(img_label).reshape(BATCH_SIZE)
            start = (start+BATCH_SIZE)%train_data_len
            end   = (end+BATCH_SIZE)%train_data_len
            if(end<start): end = start+BATCH_SIZE
            if count<=(n_StartCrossVal)*n_batch:
                lr = LEARNING_RATE_BASE
                l2_rate= REGULARAZTION_RATE
            else:
                lr = LEARNING_RATE_BASE*0.1
                l2_rate= REGULARAZTION_RATE*0.1
            summary,_t_,loss_run,pre_cls,step = sess.run([merged,train_op,loss,cls_10_softmax,global_step],
                        feed_dict={input_data:img,label:img_label,keep_prob:0.5, learning_rate:lr,L2_regu_rate:l2_rate})
            pre_label = np.argmax(pre_cls,axis=1)
            tmp_num = np.where(pre_label==img_label)[0].shape[0]
            pre_train_num += tmp_num
            #if count%400==0: print('step',step,"loss",loss_run)
            if (count+1)%n_batch==0:
                print("train acc is ",pre_train_num/(n_batch*BATCH_SIZE))
                pre_train_num = 0
            summary_writer.add_summary(summary, i)
            
            #*****************************************************************#
            ##################    cross valid      ############################
            #*****************************************************************#
            val_start = 0
            val_end = BATCH_SIZE
            val_count =0
            val_data_len = test_data.shape[0]
            val_loss_tmp = 0
            pre_right_num = 0
            val_n_batch = val_data_len//BATCH_SIZE
            if (count+1)%n_batch==0 and (count+1)//n_batch>n_StartCrossVal:
                print("start test acc of val...")
                while val_count<val_n_batch:
                    val_img =[]
                    val_img_label = []
                    for i in range(val_start,val_end):
                        i = i%val_data_len
                        img_file = os.path.join('data','train', test_data[i,1]) + '\\'+ test_data[i,2]
                        img_tmp = read_img.get_img_sample(img_file,dataAug=dataAug)
                        label_tmp = [int(test_data[i,1][1])]
                        val_img.append(img_tmp)
                        val_img_label.append(label_tmp)
                    val_img = np.array(val_img).reshape(BATCH_SIZE,img_tmp.shape[1],img_tmp.shape[2],3)
                    val_img_label = np.array(val_img_label).reshape(BATCH_SIZE)
                    val_start = (val_start+BATCH_SIZE)%train_data_len
                    val_end   = (val_end+BATCH_SIZE)%train_data_len
                    if(val_end<val_start): val_end = val_start+BATCH_SIZE
                    pre_cls,cls_loss_tmp= sess.run([cls_10_softmax,cls_loss],feed_dict={input_data:val_img,label:val_img_label,
                                        keep_prob:1.0, learning_rate:0,L2_regu_rate:0.0})
                    pre_label = np.argmax(pre_cls,axis=1)
                    num_right = np.where(pre_label==val_img_label)[0].shape[0]
                    pre_right_num += num_right
                    val_loss_tmp += cls_loss_tmp
                    val_count +=1
                accury_tmp = float(pre_right_num)/float(BATCH_SIZE*val_n_batch)
                val_loss_tmp = val_loss_tmp/val_n_batch
                if val_loss<val_loss_tmp:
                    num_acc_noupdate +=1
                    print("loss don't down",val_loss_tmp,"pre accury is",accury_tmp)
                else:
                    accury_max=accury_tmp
                    val_loss = val_loss_tmp
                    num_acc_noupdate=0
                    saver.save(sess, save_checkpoint_dir + 'model.ckpt', global_step=count+1)
                    print("save model",count+1,"loss update is",val_loss_tmp,"cross validation acc is",accury_max)
            count+=1
    ##tensorboard --logdir  /path/to/Driver_Detection 
    return test_data,pre_cls,img_label

In [ ]:
test_data,pre_cls,img_label = train(dataAug=True,checkpoint=False)
img = test_data[:,2]
subject = test_data[:,0]
classname = test_data[:,1]
dataframe = pd.DataFrame({'subject':subject,'classname':classname,'img':img})
dataframe.to_csv('cross_validation11.csv',index=False,sep=',')

train acc is  0.19691526263627354
train acc is  0.7795961347869177
train acc is  0.9193260654112984
train acc is  0.9532581764122894
train acc is  0.9686199207135778
train acc is  0.9769697720515361
start test acc of val...
save model 24216 loss update is 0.0671651583611 cross validation acc is 0.9799107142857143
train acc is  0.9816650148662042
start test acc of val...
save model 28252 loss update is 0.0510985596588 cross validation acc is 0.9860491071428571
train acc is  0.9845515361744301
start test acc of val...
save model 32288 loss update is 0.0378190444945 cross validation acc is 0.9885044642857143


In [1]:
train_c0 = ['p042','p012', 'p072', 'p024', 'p021', 'p041', 'p016', 'p045', 'p061', 'p066', 'p002', 'p015', 'p052', 'p014', 'p026', 'p056', 'p049', 'p050', 'p035', 'p075', 'p047', 'p039']
test_c0 = ['p064','p051', 'p081', 'p022']

train_c1 = [ 'p064','p051', 'p081', 'p022', 'p021', 'p041', 'p016', 'p045', 'p061', 'p066', 'p002', 'p015', 'p052', 'p014', 'p026', 'p056', 'p049', 'p050', 'p035', 'p075', 'p047', 'p039']
test_c1 = ['p042','p012', 'p072', 'p024']

train_c2 = [ 'p042','p012', 'p072', 'p024','p064','p051', 'p081', 'p022',  'p061', 'p066', 'p002', 'p015', 'p052', 'p014', 'p026', 'p056', 'p049', 'p050', 'p035', 'p075', 'p047', 'p039']
test_c2 = ['p021', 'p041', 'p016', 'p045']

train_c3 = ['p064','p051', 'p081', 'p022', 'p021', 'p041', 'p016', 'p045','p042','p012', 'p072', 'p024',   'p052', 'p014', 'p026', 'p056', 'p049', 'p050', 'p035', 'p075', 'p047', 'p039']
test_c3 = ['p061', 'p066', 'p002', 'p015']

train_c4 = [ 'p064','p051', 'p081', 'p022','p021', 'p041', 'p016', 'p045','p042','p012', 'p072', 'p024',  'p061', 'p066', 'p002', 'p015',  'p049', 'p050', 'p035', 'p075', 'p047', 'p039']
test_c4 = ['p052', 'p014', 'p026', 'p056']

train_c5 = ['p052', 'p014', 'p026', 'p056', 'p064','p051', 'p081', 'p022','p021', 'p041', 'p016', 'p045','p042','p012', 'p072', 'p024',  'p061', 'p066', 'p002', 'p015', 'p047', 'p039']
test_c5 = ['p049', 'p050', 'p035', 'p075']
print(train_c0)

['p042', 'p012', 'p072', 'p024', 'p021', 'p041', 'p016', 'p045', 'p061', 'p066', 'p002', 'p015', 'p052', 'p014', 'p026', 'p056', 'p049', 'p050', 'p035', 'p075', 'p047', 'p039']


In [3]:
driver1 = pd.read_csv('submisson/1_VGG16_cro98.csv')
driver2 = pd.read_csv('submisson/5_VGG16_loss0.07.csv')
driver3 = pd.read_csv('submisson/3_VGG16_cro98_dataAug.csv')
driver = (driver1.values[:,:10] + driver2.values[:,:10] + driver3.values[:,:10])/3
name = driver1.values[:,10]
c0 = driver[:,0]
c1 = driver[:,1]
c2 = driver[:,2]
c3 = driver[:,3]
c4 = driver[:,4]
c5 = driver[:,5]
c6 = driver[:,6]
c7 = driver[:,7]
c8 = driver[:,8]
c9 = driver[:,9]
dataframe = pd.DataFrame({'img':name,'c0':c0,'c1':c1,'c2':c2,'c3':c3,'c4':c4,'c5':c5,'c6':c6,'c7':c7,'c8':c8,'c9':c9})
dataframe.to_csv('8_VGG_test.csv',index=False,sep=',')